In [22]:
# %% [markdown]
# ## Preprocessing

# %%
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

# %%
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

# %%
# Determine the number of unique values in each column.
application_df.nunique()

# %%
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

# %%
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace =application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts()<500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()


# %%
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts

# %%
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts = application_df["CLASSIFICATION"].value_counts()
classification_value_counts[classification_value_counts > 1]

# %%
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_value_counts[classification_value_counts < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

# %%
# Convert categorical data to numeric with `pd.get_dummies`
application_type_dummies  = pd.get_dummies(application_df['APPLICATION_TYPE'], dtype='int')
classification_dummies = pd.get_dummies(application_df['CLASSIFICATION'], dtype='int')
affiliation_dummies = pd.get_dummies(application_df['AFFILIATION'], dtype='int')
usecase_dummies  = pd.get_dummies(application_df['USE_CASE'], dtype='int')
organization_dummies = pd.get_dummies(application_df['ORGANIZATION'], dtype='int')
income_dummies = pd.get_dummies(application_df['INCOME_AMT'], dtype='int')
type_dummies = pd.get_dummies(application_df['SPECIAL_CONSIDERATIONS'], dtype='int')

# Concatenate the 'application type' and 'classification' dummies with the application_df
application_df = pd.concat([application_df, application_type_dummies, classification_dummies, affiliation_dummies, usecase_dummies, organization_dummies,income_dummies, type_dummies],axis=1)

# Drop the original 'APPLICATION_TYPE' and 'CLASSIFICATION' columns from the application_df
application_df = application_df.drop(columns=['APPLICATION_TYPE', 'CLASSIFICATION','AFFILIATION','USE_CASE','ORGANIZATION','INCOME_AMT','SPECIAL_CONSIDERATIONS'])

# %%
# Split our preprocessed data into our features and target arrays
x = application_df.drop(columns=["IS_SUCCESSFUL"])
y = application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
x_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1, stratify=y)

# %%
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(X_test)

# %% [markdown]
# ## Compile, Train and Evaluate the Model

# %%
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dim = 100

nn = tf.keras.models.Sequential()

# First hidden layer,
nn.add(tf.keras.layers.Dense(units =80 , activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# %%
# Compile the model
nn.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])

# %%
# Train the model
nn.fit(x_train_scaled, y_train, epochs = 100)

# %%
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# %%
# Export our model to HDF5 file
nn.save("ApphabetSoupCharity.h5")




Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                3520      
                                                                 
 dense_13 (Dense)            (None, 30)                2430      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 9s 8ms/step - loss: 0.5708 - accuracy: 0.7231
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5556 - accuracy: 0.7293
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5528 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
